In [2]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import matplotlib.pyplot as plt
import seaborn as sns

import xgboost as xgb

%matplotlib inline

### XGB Benchmark
Gets 0.984 LB after 200 rounds (Could be much better)

In [16]:
train_dat = pd.read_csv('train.csv')
test_dat = pd.read_csv('test.csv')
# Target encoding
lbl = preprocessing.LabelEncoder()
lbl.fit(list(train_dat.target.values))
train_dat.target = lbl.transform(list(train_dat.target.values))
# Extract labels
train_labels = train_dat.target
# Drop labels
train_dat = train_dat.drop('target',axis=1)
# Drop id columns
train_dat = train_dat.drop('id',axis=1)
test_dat = test_dat.drop('id',axis=1)

In [17]:
train_dat.head()

,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,feat_10,...,feat_84,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93
0,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,1,6,1,5,0,0,1,...,22,0,1,2,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0


In [36]:
# Hold a set to test on
dev_stop = int( len(train_dat)*0.90 )
xg_train = xgb.DMatrix( train_dat.iloc[:dev_stop], label=train_labels[:dev_stop])
xg_test = xgb.DMatrix( train_dat.iloc[dev_stop:], label=train_labels[dev_stop:])

param = {   'objective': 'multi:softprob',
            'eval_metric': 'mlogloss',
            'num_class': train_labels.nunique(),
            'nthread': 2,
            'max_depth': 8,
            'colsample_bytree':1.0,
            'subsample':1.0,
            'eta':0.01,
        }
watchlist = [ (xg_train,'train'), (xg_test, 'test') ]
num_round = 500

bst = xgb.train(param, xg_train, num_round, watchlist)

[0]	train-mlogloss:2.170711	test-mlogloss:2.194244
[1]	train-mlogloss:2.144315	test-mlogloss:2.191535
[2]	train-mlogloss:2.119371	test-mlogloss:2.189201
[3]	train-mlogloss:2.094864	test-mlogloss:2.187071
[4]	train-mlogloss:2.071118	test-mlogloss:2.185137
[5]	train-mlogloss:2.048998	test-mlogloss:2.183191
[6]	train-mlogloss:2.026258	test-mlogloss:2.181630
[7]	train-mlogloss:2.004854	test-mlogloss:2.180095
[8]	train-mlogloss:1.984127	test-mlogloss:2.178865
[9]	train-mlogloss:1.963922	test-mlogloss:2.177621
[10]	train-mlogloss:1.944198	test-mlogloss:2.176541
[11]	train-mlogloss:1.924741	test-mlogloss:2.175563
[12]	train-mlogloss:1.906176	test-mlogloss:2.174733
[13]	train-mlogloss:1.887641	test-mlogloss:2.174170
[14]	train-mlogloss:1.870084	test-mlogloss:2.173592
[15]	train-mlogloss:1.852251	test-mlogloss:2.173179
[16]	train-mlogloss:1.835254	test-mlogloss:2.172979
[17]	train-mlogloss:1.818641	test-mlogloss:2.172778
[18]	train-mlogloss:1.802349	test-mlogloss:2.172712
[19]	train-mlogloss:1.

In [37]:
xg_pred = xgb.DMatrix( test_dat )

# predict on test set
preds = bst.predict(xg_pred)

samp = pd.read_csv('sampleSubmission.csv')
preds = pd.DataFrame(preds, index=samp.id.values, columns=samp.columns[1:])
preds.to_csv('submission.csv', index_label='id')